In [124]:
import pandas as pd
import numpy as np
eps = np.finfo(float).eps
from numpy import log2
import math
data=pd.read_csv('../input_data/decision_Tree/train.csv')
data,testdata= np.split(data,[int(0.80*len(data))])

In [125]:
class Node:
    def __init__(self,split,rows,leaf,numeric):
        self.isleaf=leaf
        self.rows=rows
        self.isnumeric=numeric
#         print("rows", rows)
        yes,no = maxfreq(rows)
       
        if yes==0 or no ==0:
            self.isleaf=True
        
        if self.isleaf==True:
            if yes>no:
                self.value=1
            else:
                self.value=0
            self.children={}
        else:
            self.value=split #name of the attribute is the value of the node
            if self.isnumeric:
                self.children,self.minsplit= partitionnumerical(rows,split)
            else:
                self.children= partitioncategorical(rows,split) #all children of this attribute in a dictionary
        print (self.value,"Node Value")
        return
        

In [126]:
def maxfreq(rows):
    yes=0
    no=0
    for x in range(len(rows)):
        if rows.iloc[x]['left']==1:
            yes+=1
        else:
            no+=1
    return yes,no

In [127]:
def partitionnumerical(data, col):
#     print("col",col)
    countdict={}
    minent=float('inf')
    data.sort_values([col],axis=0,ascending=True,inplace=True)
    uniquevalues = data[col].unique()
    for value in uniquevalues:
        ent=0
        data1=data[data[col]<=value]
        data2=data[data[col]>value]
        data1stats = matchlabel(data1)
        data2stats = matchlabel(data2)
        
        ans=0
        for op in data1stats:
            val=data1stats[op]
            ans-= float(val)/float(len(data1)) * math.log((float(val)/float(len(data1))),2)
        
        ent+=ans*(len(data1)/len(data))
       
        ans=0
        for op in data2stats:
            val=data2stats[op]
            ans-= float(val)/float(len(data2)) * math.log((float(val)/float(len(data2))),2)
        
        ent+=ans*(len(data2)/len(data))
    
        if ent<minent:
            minent=ent
            minsplit=value
    
#     print("categorical ent",minent)
#     print("minsplit",minsplit)
#     print("minsplit at",minsplit)
    countdict[0]=data[data[col]<=minsplit]
    countdict[1]=data[data[col]>minsplit]
    return countdict,minsplit


In [128]:
# Partition function for numerical data while using misclassification

In [129]:
def mispartitionnumerical(data, col):
#     print("col",col)
    countdict={}
    minent=float('inf')
    data.sort_values([col],axis=0,ascending=True,inplace=True)
    uniquevalues = data[col].unique()
    for value in uniquevalues:
        ent=0
        data1=data[data[col]<=value]
        data2=data[data[col]>value]
        data1stats = matchlabel(data1)
        data2stats = matchlabel(data2)
        
        ans=0
        for op in data1stats:
            val=data1stats[op]
            ans+= min(float(val)/float(len(data1)),(1-float(val)/float(len(data1))))
        
        ent+=ans*(len(data1)/len(data))
       
        ans=0
        for op in data2stats:
            val=data2stats[op]
            ans+= min(float(val)/float(len(data2)),(1-float(val)/float(len(data2))))
        
        ent+=ans*(len(data2)/len(data))
    
        if ent<minent:
            minent=ent
            minsplit=value
    
#     print("categorical ent",minent)
#     print("minsplit",minsplit)
#     print("minsplit at",minsplit)
    countdict[0]=data[data[col]<=minsplit]
    countdict[1]=data[data[col]>minsplit]
    return countdict,minsplit


In [130]:
#partitions the rows into multiple groups based on the column passed
def partitioncategorical(data,col):
    countdict={}
    uniquevalues = data[col].unique()
    for value in uniquevalues:
        countdict[value]= data[data[col]==value]
#     print (countdict)
    return countdict

In [131]:
#matches the passed rows to count the number of yes and no in the rows 
def matchlabel(data):
    stats={}
    values, valuecount = np.unique(data['left'],return_counts=True)
    for i in range(len(values)):
        stats[values[i]]=valuecount[i]
    return stats

In [132]:
def isnumeric(col):
    return data[col].dtype.kind in 'bifc'

In [133]:
#calculate the entropy of the passed column
def entropy(rows,col):
    entro=0
    
    if isnumeric(col):
#         print(col,"is numeric")
        countdict,minsplit =partitionnumerical(rows,col)
    else:
        countdict= partitioncategorical(rows,col)
    
#     print("countdict",countdict)
    for key in countdict:
        rowgroup=countdict[key]
        rowgroupstats=matchlabel(rowgroup)
        ans=0
#         print(rowgroupstats)
        for op in rowgroupstats:
            value=rowgroupstats[op]
            ans-= float(value)/float(len(rowgroup)) * math.log((float(value)/float(len(rowgroup))),2)
        
        entro+=ans*(len(rowgroup)/len(rows))
    return entro

In [134]:
def ginientropy(rows,col):
    entro=0
    
    if isnumeric(col):
#         print(col,"is numeric")
        countdict,minsplit = partitionnumerical(rows,col)
    else:
        countdict= partitioncategorical(rows,col)
    
#     print("countdict",countdict)
    for key in countdict:
        rowgroup=countdict[key]
        rowgroupstats=matchlabel(rowgroup)
        ans=0
#         print(rowgroupstats)
        for op in rowgroupstats:
            value=rowgroupstats[op]
            ans+= 2 * (float(value)/float(len(rowgroup)) * (1- float(value)/float(len(rowgroup))))
        
        entro+=ans*(len(rowgroup)/len(rows))
    return entro

In [135]:
def misentropy(rows,col):
    entro=0
    
    if isnumeric(col):
#         print(col,"is numeric")
        countdict,minsplit= mispartitionnumerical(rows,col)
    else:
        countdict= partitioncategorical(rows,col)
    
#     print("countdict",countdict)
    for key in countdict:
        rowgroup=countdict[key]
        rowgroupstats=matchlabel(rowgroup)
        ans=0
#         print(rowgroupstats)
        for op in rowgroupstats:
            value=rowgroupstats[op]
            ans+= min(float(value)/float(len(rowgroup)),(1- float(value)/float(len(rowgroup))))
        
        entro+=ans*(len(rowgroup)/len(rows))
    return entro

In [136]:
def totalgini(data):
    countdict= partitioncategorical(data,'left')
    total = 0
    for key in countdict:
        value=len(countdict[key])
        total+= 2 * (float(value)/float(len(data)) * (1-float(value)/float(len(data))))
#     print (total)
    return total

In [137]:
totalgini(data)

0.7231765365298979

In [138]:
def totalmis(data):
    countdict=partitioncategorical(data,'left')
    total = 0
    for key in countdict:
        value=len(countdict[key])
        total+= min(float(value)/float(len(data)),(1-float(value)/float(len(data))))
#     print (total)
    return total

In [139]:
totalmis(data)

0.47385984427141264

In [140]:
def totalentropy(data):
    countdict=partitioncategorical(data,'left')
    total = 0
    for key in countdict:
        value=len(countdict[key])
        total-= float(value)/float(len(data)) * math.log((float(value)/float(len(data))),2)
    return total

In [141]:
totalentropy(data)

0.789897462686135

In [142]:
def informationgain(data,total):
#     print("length of data", len(data))
    maxinfogain=0
    attr=0
    infogain=0
    for col in data:
        if col=='left': 
            continue
        ent=entropy(data,col)
#         print("Total and entropy",total,ent)
        infogain=total-ent
#         print ("infogain",infogain)
        if infogain>maxinfogain:
#             print(maxinfogain)
            maxinfogain=infogain
            attr=col
    
    print ("maxinfogain",maxinfogain)
    return maxinfogain,attr

In [143]:
level=0
def buildTree(data):
#     global level
    total=totalentropy(data)
#     print(total,"total")
    gain, split = informationgain(data,total)
    print("split", split)
#     level+=1
    if gain<=0:
        return Node(split,data,True,False)
    
    root = Node(split,data,False,isnumeric(split))
#     print( "root->children",root.children)
    for child in root.children:
#         print(root.children[child].drop(columns=[split]))
        if(isnumeric(split)):
            root.children[child]=buildTree(root.children[child])
        else:
            root.children[child]=buildTree(root.children[child].drop(columns=[split]))
        
#     level-=1
    return root
        

In [144]:
rootentropy =buildTree(data)
# print(root.isleaf)

maxinfogain 0.19436853577423685
split satisfaction_level
satisfaction_level Node Value


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


maxinfogain 0.1865934358814455
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0.5602794945111382
split number_project
number_project Node Value
maxinfogain 0.22504767583407675
split last_evaluation
last_evaluation Node Value
maxinfogain 0.09992214726926568
split last_evaluation
last_evaluation Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.06650229651488529
split average_montly_hours
average_montly_hours Node Value
maxinfogain 0.04964613899663595
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.03334304532716438
split average_montly_hours
average_montly_hours Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.004095709907173353
split salary
salary Node Value
maxinfogain 0.0037091437078117795
split sales
sales Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0.008149652590823522
split average_montly_hours
average_montly_hours Node Value
maxinfogain

maxinfogain 0.9709505944546686
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 1.0
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.12601435899817987
split last_evaluation
last_evaluation Node Value
maxinfogain 0.03957936040923704
split last_evaluation
last_evaluation Node Value
maxinfogain 0.07749938426924849
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0.8112781244591328
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.3166890883150208
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 1.0
split number_project
number_project Node Value
maxinfoga

maxinfogain 1.0
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0.02682672523718896
split average_montly_hours
average_montly_hours Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.3372900666170139
split average_montly_hours
average_montly_hours Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.014259585635922604
split last_evaluation
last_evaluation Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.1063879557629547
split number_project
number_project Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.8112781244591328
split average_montly_hours
average_montly_hours Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.031983643992597094
split average_montly_hours
average_montly_hours Node Value
maxinfogain 0
split 0

maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.3351291423194559
split last_evaluation
last_evaluation Node Value
maxinfogain 0.05700902421203441
split sales
sales Node Value
maxinfogain 0.07794434474845494
split average_montly_hours
average_montly_hours Node Value
maxinfogain 0.2283751837837329
split last_evaluation
last_evaluation Node Value
maxinfogain 0.39124356362925566
split average_montly_hours
average_montly_hours Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.2576788051033316
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 1.0
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.2327394880479417
split salary
salary Node Value
maxinfogain 0.72192809488736

In [90]:
rootginientropy = buildTree(data)

maxinfogain 0.2096986463517283
split satisfaction_level
satisfaction_level Node Value


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


maxinfogain 0.21980933010978787
split number_project
number_project Node Value
maxinfogain 0.21163201719343927
split last_evaluation
last_evaluation Node Value
maxinfogain 0.07811957588190653
split last_evaluation
last_evaluation Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.03920465239371888
split average_montly_hours
average_montly_hours Node Value
maxinfogain 0.026159172189128325
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0.4375
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.017503524160075216
split average_montly_hours
average_montly_hours Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.0001464918334482708
split salary
salary Node Value
maxinfogain 0.00010133617695056406
split last_evaluation
last_evaluation Node Value
maxinfogain 3.431446540880928e-05
split average_montly_hours
average_montly_hours Node Value
maxinfogain 0
split 0
1 Node Val

maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.01728994569333364
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0.14775916704391134
split number_project
number_project Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.28063492063492057
split sales
sales Node Value
maxinfogain 0.435374149659864
split number_project
number_project Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.888888888888889
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.888888888888889
split last_evaluation
last_evaluation Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0.5555555555555556
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 1.0
split last_evaluation
last_evaluation Node Va

maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.00035399888546338437
split time_spend_company
time_spend_company Node Value
maxinfogain 4.940020228954081e-05
split sales
sales Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.00018426998578979434
split average_montly_hours
average_montly_hours Node Value
maxinfogain 0.06858710562414277
split satisfaction_level
satisfaction_level Node Value
maxinfogain 1.0
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.00026797683225009833
split last_evaluation
last_evaluation Node Value
maxinfogain 0.0002257345477857317
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0.28402366863905315
split average_montly_hours
average_montly_hours Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0.00

maxinfogain 0.0010387398893145494
split average_montly_hours
average_montly_hours Node Value
maxinfogain 0.13317479191438755
split average_montly_hours
average_montly_hours Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.0564373897707231
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0.05442176870748308
split average_montly_hours
average_montly_hours Node Value
maxinfogain 0.888888888888889
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.888888888888889
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.1836547291092745
split last_evaluation
last_evaluation Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.6887052341597797
split sa

maxinfogain 0.13442892185246397
split number_project
number_project Node Value
maxinfogain 0.75
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0.03288818626839185
split last_evaluation
last_evaluation Node Value
maxinfogain 0.03999999999999998
split last_evaluation
last_evaluation Node Value
maxinfogain 0.888888888888889
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.2998027613412228
split sales
sales Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0.888888888888889
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.888888888888889
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0
split 0


In [111]:
rootmisentropy =buildTree(data)

maxinfogain 0.12191323692992201
split satisfaction_level
satisfaction_level Node Value


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


maxinfogain 0
split 0
1 Node Value
maxinfogain 0.10803127221037673
split number_project
number_project Node Value
maxinfogain 0.47496423462088705
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0.12623274161735704
split last_evaluation
last_evaluation Node Value
maxinfogain 0.04237288135593223
split average_montly_hours
average_montly_hours Node Value
maxinfogain 0.023861171366594304
split last_evaluation
last_evaluation Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.015367727771679513
split satisfaction_level
satisfaction_level Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 0.008849557522123911
split average_montly_hours
average_montly_hours Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 1.474514954580286e-17
split last_evaluation
last_evaluation Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0
split 0
1 Node Value
maxinfogain 0
split 0
0 Node Value
maxinfogain 1.3877787807814457e-17
split average_montly_hours
average_mo

In [112]:
def findlabel(row, ptr):
    while ptr.isleaf==False:
        if ptr.isnumeric==False:
            try:
                value=row[ptr.value]
#                 print(ptr.children)
                ptr=ptr.children[value]
            except:
                return 0
        else:
            try:
                value=row[ptr.value]
                if value<=ptr.minsplit:
                    ptr=ptr.children[0]
                else:
                    ptr=ptr.children[1]
            except:
                return 0
    
    return ptr.value

In [113]:
def calculate(fp,fn,tp,tn,wrong,correct):
    accuracy=correct/(wrong+correct)
    recall=tp/(tp+fn)
    precision=tp/(tp+fp)
    f1score=(2/(1/precision)+(1/recall))
    return accuracy,recall,precision,f1score

In [117]:
def predict(data, ptr):
    correct=0
    wrong=0
    fp=0
    tp=0
    fn=0
    tn=0
    for i in range(0,len(data)):
        row=data.iloc[i]
        predictlabel=findlabel(row,ptr)
        if predictlabel==row['left']:
            if predictlabel==1:
                tp+=1
            else:
                tn+=1
            correct+=1
        else:
            if predictlabel==0:
                fn+=1
            else:
                fp+=1
            wrong+=1
    print(fp,fn,tp,tn,wrong,correct)
    accuracy,recall,precision,f1score=calculate(fp,fn,tp,tn,wrong,correct)
    return accuracy,recall,precision,f1score

In [145]:
accuracy,recall,precision,f1score = predict(testdata, rootentropy)
print(accuracy,recall,precision,f1score)

27 23 522 1676 50 2198
0 0.9577981651376147 0.9508196721311475 2.9457006469442875


In [119]:
accuracy,recall,precision,f1score = predict(testdata, rootmisentropy)
print(accuracy,recall,precision,f1score)

5 51 494 1698 56 2192
0 0.9064220183486239 0.9899799599198397 3.0831987862364407


In [120]:
accuracy,recall,precision,f1score = predict(testdata, rootginientropy)
print(accuracy,recall,precision,f1score)

34 18 527 1669 52 2196
0 0.9669724770642202 0.9393939393939394 2.912943476510839
